# <center>RumbleDB sandbox</center>


This is a RumbleDB sandbox that allows you to play with simple JSONiq queries.

It is a jupyter notebook that you can also download and execute on your own machine, but if you arrived here from the RumbleDB website, it is likely to be shown within Google's Colab environment.

To get started, you first need to execute the cell below to activate the RumbleDB magic (you do not need to understand what it does, this is just initialization Python code).

In [88]:
!pip install rumbledb
%load_ext rumbledb
%env RUMBLEDB_SERVER=http://public.rumbledb.org:9090/jsoniq

The rumbledb extension is already loaded. To reload it, use:
  %reload_ext rumbledb
env: RUMBLEDB_SERVER=http://public.rumbledb.org:9090/jsoniq


By default, this notebook uses a small public backend provided by us. Each query runs on just one machine that is very limited in CPU: one core and memory: 1GB, and with only the http scheme activated. This is sufficient to discover RumbleDB and play a bit, but of course is not intended for any production use. If you need to use RumbleDB in production, you can use it with an installation of Spark either on your machine or on a cluster.

This sandbox backend may occasionally break, especially if too many users use it at the same time, so please bear with us! The system is automatically restarted every day so, if it stops working, you can either try again in 24 hours or notify us.


It is straightforward to execute your own RumbleDB server on your own Spark cluster (and then you can make full use of all the input file systems and file formats). In this case, just replace the above server with your own hostname and port. Note that if you run RumbleDB as a server locally, you will also need to download and use this notebook locally rather than in this Google Colab environment as, obviously, your personal computer cannot be accessed from the Web.

Now we are all set! You can now start reading and executing the JSONiq queries as you go, and you can even edit them!

## The dataset

The dataset contains products:

In [92]:
%%jsoniq

json-file("https://www.rumbledb.org/samples/products-small.json")[position() le 10]

Took: 1.8291950225830078 ms
{"product": "blender", "store-number": 20, "quantity": 920}
{"product": "shirt", "store-number": 39, "quantity": 839}
{"product": "tv", "store-number": 58, "quantity": 758}
{"product": "toaster", "store-number": 77, "quantity": 677}
{"product": "socks", "store-number": 96, "quantity": 596}
{"product": "phone", "store-number": 15, "quantity": 515}
{"product": "broiler", "store-number": 34, "quantity": 434}
{"product": "blender", "store-number": 53, "quantity": 353}
{"product": "shirt", "store-number": 72, "quantity": 272}
{"product": "tv", "store-number": 91, "quantity": 191}


As well as contries:

In [90]:
%%jsoniq

json-file("https://www.rumbledb.org/samples/countries.json")[position() le 10]

Took: 0.6905701160430908 ms
{"sid": 1, "country": "Afghanistan"}
{"sid": 2, "country": "land Islands"}
{"sid": 3, "country": "Albania"}
{"sid": 4, "country": "Algeria"}
{"sid": 5, "country": "American Samoa"}
{"sid": 6, "country": "AndorrA"}
{"sid": 7, "country": "Angola"}
{"sid": 8, "country": "Anguilla"}
{"sid": 9, "country": "Antarctica"}
{"sid": 10, "country": "Antigua and Barbuda"}


## For clauses

For clauses bind their variable in turn to each item of the provided expression. Here is an example:

In [91]:
%%jsoniq
for$x in 1 to 10
return
  {
    "number": $x,
    "square": $x * $x
  }

Took: 0.05975675582885742 ms
{"number": 1, "square": 1}
{"number": 2, "square": 4}
{"number": 3, "square": 9}
{"number": 4, "square": 16}
{"number": 5, "square": 25}
{"number": 6, "square": 36}
{"number": 7, "square": 49}
{"number": 8, "square": 64}
{"number": 9, "square": 81}
{"number": 10, "square": 100}


In the above query, the variable $x is bound with 1, then with 2, then with 3, etc, and finally with 10. It is always bound with a sequence of exactly one item. It is, however, possible to bind it with an empty sequence if the expression returns no items. This is done with “allowing empty”.

In [13]:
%%jsoniq
for $x allowing empty in () return count($x)

Took: 0.01749706268310547 ms
0


Note that, without “allowing empty”, if the expression in the for clause evaluates to an empty sequence, the variable would not bind to anything at all and the FLWOR expression would simply return an empty sequence.

In [14]:
%%jsoniq
for $x in () return count($x)

Took: 0.01473093032836914 ms


Each variable binding is also more generally called a tuple. In this above examples, there is only one variable binding in each tuple (x), but it is possible to build larger tuples with more clauses. For example, this FLWOR expression involves two for clauses. The tuples after the first for clause and before the second one only bind variable x (to 1, then to 2, then to 3), but the tuple after the second for clause and before the return clause bind variables x and y. There are six tuples in total, because the second for clause expands each incoming tuple to zero, one or more tuples (think of a flatMap transformation in Spark for an analogy).

In [16]:
%%jsoniq
for$x in 1 to 3
for $y in 1 to $x
return [ $x, $y ]

Took: 0.017578125 ms
[1, 1]
[2, 1]
[2, 2]
[3, 1]
[3, 2]
[3, 3]


Now if we use our small example dataset, we can iterate on all objects, say, products:

In [19]:
%%jsoniq
for $product in json-file("http://www.rumbledb.org/samples/products-small.json")
return $product.product

Took: 2.0165750980377197 ms
"Warning! The output sequence contains 100000 items but its materialization was capped at 200 items. This value can be configured with the result-size parameter in the query string of the HTTP request."
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"b

It can thus be seen that the for clause is akin to the FROM clause in SQL, and the return is akin to the SELECT clause.
Projection in JSONiq can be made with a project() function call, with the keys to keep:

In [24]:
%%jsoniq
for $product in json-file("http://www.rumbledb.org/samples/products-small.json")
return project($product, ("product", "store-number"))

Took: 1.7290990352630615 ms
"Warning! The output sequence contains 100000 items but its materialization was capped at 200 items. This value can be configured with the result-size parameter in the query string of the HTTP request."
{"product": "blender", "store-number": 20}
{"product": "shirt", "store-number": 39}
{"product": "tv", "store-number": 58}
{"product": "toaster", "store-number": 77}
{"product": "socks", "store-number": 96}
{"product": "phone", "store-number": 15}
{"product": "broiler", "store-number": 34}
{"product": "blender", "store-number": 53}
{"product": "shirt", "store-number": 72}
{"product": "tv", "store-number": 91}
{"product": "toaster", "store-number": 10}
{"product": "socks", "store-number": 29}
{"product": "phone", "store-number": 48}
{"product": "broiler", "store-number": 67}
{"product": "blender", "store-number": 86}
{"product": "shirt", "store-number": 5}
{"product": "tv", "store-number": 24}
{"product": "toaster", "store-number": 43}
{"product": "socks", "sto

Let us look closer at the key called "type". What values does it take? We can use dot-based navigation to navigate down to these values. This will work nicely on the entire dataset.

Took: 1.7029812335968018 ms
"Warning! The output sequence contains 100000 items but its materialization was capped at 200 items. This value can be configured with the result-size parameter in the query string of the HTTP request."
{"store-number": 20}
{"store-number": 39}
{"store-number": 58}
{"store-number": 77}
{"store-number": 96}
{"store-number": 15}
{"store-number": 34}
{"store-number": 53}
{"store-number": 72}
{"store-number": 91}
{"store-number": 10}
{"store-number": 29}
{"store-number": 48}
{"store-number": 67}
{"store-number": 86}
{"store-number": 5}
{"store-number": 24}
{"store-number": 43}
{"store-number": 62}
{"store-number": 81}
{"store-number": 100}
{"store-number": 19}
{"store-number": 38}
{"store-number": 57}
{"store-number": 76}
{"store-number": 95}
{"store-number": 14}
{"store-number": 33}
{"store-number": 52}
{"store-number": 71}
{"store-number": 90}
{"store-number": 9}
{"store-number": 28}
{"store-number": 47}
{"store-number": 66}
{"store-number": 85}
{"store-number

It is possible to implement a join with a sequence of two for clauses and a predicate (note that newlines in JSONiq are irrelevant, so we spread the for clause on two lines in order to fit the query on this page):

In [29]:
%%jsoniq
for $product in json-file("http://www.rumbledb.org/samples/products-small.json")
for $store in json-file("http://www.rumbledb.org/samples/countries.json")
                    [$$.sid eq $product.store-number]
return {
  "product" : $product.product,
  "country" : $store.country
}

Took: 11.249583959579468 ms
"Warning! The output sequence contains 100000 items but its materialization was capped at 200 items. This value can be configured with the result-size parameter in the query string of the HTTP request."
{"product": "socks", "country": "Botswana"}
{"product": "socks", "country": "Bhutan"}
{"product": "broiler", "country": "Botswana"}
{"product": "broiler", "country": "Bhutan"}
{"product": "shirt", "country": "Botswana"}
{"product": "shirt", "country": "Bhutan"}
{"product": "toaster", "country": "Botswana"}
{"product": "toaster", "country": "Bhutan"}
{"product": "phone", "country": "Botswana"}
{"product": "phone", "country": "Bhutan"}
{"product": "blender", "country": "Botswana"}
{"product": "blender", "country": "Bhutan"}
{"product": "tv", "country": "Botswana"}
{"product": "tv", "country": "Bhutan"}
{"product": "socks", "country": "Botswana"}
{"product": "socks", "country": "Bhutan"}
{"product": "broiler", "country": "Botswana"}
{"product": "broiler", "count

It is possible to implement a join with a sequence of two for clauses and a predicate (note that newlines in JSONiq are irrelevant, so we spread the for clause on two lines in order to fit the query on this page):

In [39]:
%%jsoniq
for $product in json-file("http://www.rumbledb.org/samples/products-small.json")
for $store allowing empty in json-file("http://www.rumbledb.org/samples/countries.json")
                       [$$.sid eq $product.store-number]
return {
  "product" : $product.product,
  "country" : [ $store.country ]
}

Took: 12.377734184265137 ms
"Warning! The output sequence contains 100000 items but its materialization was capped at 200 items. This value can be configured with the result-size parameter in the query string of the HTTP request."
{"product": "blender", "country": ["Barbados"]}
{"product": "shirt", "country": ["Canada"]}
{"product": "tv", "country": ["Czech Republic"]}
{"product": "toaster", "country": ["French Southern Territories"]}
{"product": "socks", "country": ["Holy See (Vatican City State)"]}
{"product": "phone", "country": ["Austria"]}
{"product": "broiler", "country": ["Bulgaria"]}
{"product": "blender", "country": ["Costa Rica"]}
{"product": "shirt", "country": ["Fiji"]}
{"product": "tv", "country": ["Guinea"]}
{"product": "toaster", "country": ["Antigua and Barbuda"]}
{"product": "socks", "country": ["Botswana"]}
{"product": "phone", "country": ["Colombia"]}
{"product": "broiler", "country": ["Eritrea"]}
{"product": "blender", "country": ["Grenada"]}
{"product": "shirt", "c

In the case of the last product, no matching record in stores.json is found and store is bound to the empty sequence for that tuple. When constructing the object in the return clause’s expression, the empty sequence obtained from store.country is automatically replaced with a null value (because an object value cannot be empty). But if we add an array constructor around the country, we will notice the empty sequence:

In [40]:
%%jsoniq
for $product in json-file("http://www.rumbledb.org/samples/products-small.json")
for $store allowing empty in json-file("http://www.rumbledb.org/samples/countries.json")
                       [$$.sid eq $product.store-number]
return {
  "product" : $product.product,
  "country" : [ $store.country ]
}

Took: 5.071000099182129 ms
"Warning! The output sequence contains 100000 items but its materialization was capped at 200 items. This value can be configured with the result-size parameter in the query string of the HTTP request."
{"product": "blender", "country": ["Barbados"]}
{"product": "shirt", "country": ["Canada"]}
{"product": "tv", "country": ["Czech Republic"]}
{"product": "toaster", "country": ["French Southern Territories"]}
{"product": "socks", "country": ["Holy See (Vatican City State)"]}
{"product": "phone", "country": ["Austria"]}
{"product": "broiler", "country": ["Bulgaria"]}
{"product": "blender", "country": ["Costa Rica"]}
{"product": "shirt", "country": ["Fiji"]}
{"product": "tv", "country": ["Guinea"]}
{"product": "toaster", "country": ["Antigua and Barbuda"]}
{"product": "socks", "country": ["Botswana"]}
{"product": "phone", "country": ["Colombia"]}
{"product": "broiler", "country": ["Eritrea"]}
{"product": "blender", "country": ["Grenada"]}
{"product": "shirt", "co

## Let clauses

As seen before, the let clause can be used to bind a variable with any sequence of items, also more than one. FLWOR expressions with just a cascade of let clauses are quite popular.

In [41]:
%%jsoniq
let $x:=2
return $x * $x

Took: 0.016152143478393555 ms
4


However, let clauses can also appear after other clauses, for example, after a for clause. Then, they will bind a sequence of items for each previous binding (tuple), like so:

In [42]:
%%jsoniq
for $x in 1 to 10
let $square := $x * $x
return {
    "number": $x,
    "square": $square
  }

Took: 0.015926837921142578 ms
{"number": 1, "square": 1}
{"number": 2, "square": 4}
{"number": 3, "square": 9}
{"number": 4, "square": 16}
{"number": 5, "square": 25}
{"number": 6, "square": 36}
{"number": 7, "square": 49}
{"number": 8, "square": 64}
{"number": 9, "square": 81}
{"number": 10, "square": 100}


In the above example, $square is only bound with one item. Here is another example where it is bound with more than one:


In [43]:
%%jsoniq
for $x in 1 to 10
let $square-and-cube := ($x * $x, $x * $x * $x)
return
  {
    "number": $x,
    "square": $square-and-cube[1],
    "cube": $square-and-cube[2]
  }

Took: 0.020356178283691406 ms
{"number": 1, "square": 1, "cube": 1}
{"number": 2, "square": 4, "cube": 8}
{"number": 3, "square": 9, "cube": 27}
{"number": 4, "square": 16, "cube": 64}
{"number": 5, "square": 25, "cube": 125}
{"number": 6, "square": 36, "cube": 216}
{"number": 7, "square": 49, "cube": 343}
{"number": 8, "square": 64, "cube": 512}
{"number": 9, "square": 81, "cube": 729}
{"number": 10, "square": 100, "cube": 1000}


In [44]:
%%jsoniq
for $x in 1 to 10
for $square-or-cube in ($x * $x, $x * $x * $x)
return
   {
     "number": $x,
     "square or cube": $square-or-cube
   }

Took: 0.019574880599975586 ms
{"number": 1, "square or cube": 1}
{"number": 1, "square or cube": 1}
{"number": 2, "square or cube": 4}
{"number": 2, "square or cube": 8}
{"number": 3, "square or cube": 9}
{"number": 3, "square or cube": 27}
{"number": 4, "square or cube": 16}
{"number": 4, "square or cube": 64}
{"number": 5, "square or cube": 25}
{"number": 5, "square or cube": 125}
{"number": 6, "square or cube": 36}
{"number": 6, "square or cube": 216}
{"number": 7, "square or cube": 49}
{"number": 7, "square or cube": 343}
{"number": 8, "square or cube": 64}
{"number": 8, "square or cube": 512}
{"number": 9, "square or cube": 81}
{"number": 9, "square or cube": 729}
{"number": 10, "square or cube": 100}
{"number": 10, "square or cube": 1000}


A let clause outputs exactly one outgoing tuple for each incoming tuple (think of a map transformation in Spark). Unlike the for clause, it does not modify the number of tuples.
Let us now showcase the use of a let clause with our dataset.
Now if we use our small example dataset, we can iterate on all objects, say, products:

In [45]:
%%jsoniq
for $product in json-file("http://www.rumbledb.org/samples/products-small.json")
let $type := $product.product
return $type

Took: 0.6827859878540039 ms
"Warning! The output sequence contains 100000 items but its materialization was capped at 200 items. This value can be configured with the result-size parameter in the query string of the HTTP request."
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"blender"
"shirt"
"tv"
"toaster"
"socks"
"phone"
"broiler"
"b

Let clauses also allow for joining the two datasets and denormalizing them by nesting the stores into the products. This would be consider- ably more difficult to do with (Spark) SQL, even with extensions. The results are pretty-printed for ease of read.

In [66]:
%%jsoniq
for $store in  json-file("http://www.rumbledb.org/samples/countries.json")
let $product :=  json-file("http://www.rumbledb.org/samples/products-small.json")
                [$store.sid eq $$.store-number]
return {
  "store" : $store.country,
  "available products" : [
   distinct-values($product.product)
 ]
}


Took: 1.1218500137329102 ms
{"store": "Angola", "available products": ["toaster", "phone", "blender", "tv", "socks", "broiler", "shirt"]}
{"store": "AndorrA", "available products": ["toaster", "phone", "blender", "tv", "socks", "broiler", "shirt"]}
{"store": "Antarctica", "available products": ["toaster", "phone", "blender", "tv", "socks", "broiler", "shirt"]}
{"store": "American Samoa", "available products": ["shirt", "toaster", "phone", "blender", "tv", "socks", "broiler"]}
{"store": "Afghanistan", "available products": ["shirt", "toaster", "phone", "blender", "tv", "socks", "broiler"]}
{"store": "Antigua and Barbuda", "available products": ["toaster", "phone", "blender", "tv", "socks", "broiler", "shirt"]}
{"store": "Albania", "available products": ["shirt", "toaster", "phone", "blender", "tv", "socks", "broiler"]}
{"store": "Armenia", "available products": ["phone", "blender", "tv", "socks", "broiler", "shirt", "toaster"]}
{"store": "Anguilla", "available products": ["toaster", "ph

## Where clauses


Where clauses are used to filter variable bindings (tuples) based on a predicate on these variables. They are the equivalent to a WHERE clause in SQL.
This is a simple example of its use in conjunction with a for clause:

In [67]:
%%jsoniq
for $x in 1 to 10
where $x gt 7
return {
  "number": $x,
  "square": $x * $x
}

Took: 0.014545917510986328 ms
{"number": 8, "square": 64}
{"number": 9, "square": 81}
{"number": 10, "square": 100}


A where clause can appear anywhere in a FLWOR expression, ex- cept that it cannot be the first clause (always for or let) or the last clause (always return).

In [68]:
%%jsoniq
for $x in 1 to 10
let $square := $x * $x
where $square > 60
for $y in $square to $square + 1
return {
  "number": $x,
  "y": $y
}

Took: 0.021066904067993164 ms
{"number": 8, "y": 64}
{"number": 8, "y": 65}
{"number": 9, "y": 81}
{"number": 9, "y": 82}
{"number": 10, "y": 100}
{"number": 10, "y": 101}


A where clause always outputs a subset (or all) of its incoming tuples, without any alteration. In the case that the predicate always evaluates to true, it forwards all tuples, as if there had been no where clause at all. In the case that the predicate always evaluates to false, it outputs no tuple and the FLWOR expression will then return the empty sequence, with no need to further evaluate any of the remaining clauses.
Here is another example of use of the where clause with our datasets:

In [70]:
%%jsoniq
for $product in json-file("http://www.rumbledb.org/samples/products-small.json")
let $store := json-file("http://www.rumbledb.org/samples/countries.json")
                [$$.sid eq $product.store-number]
where $store.country = "Germany"
return $product.product

Took: 1.835021734237671 ms
"Warning! The output sequence contains 1000 items but its materialization was capped at 200 items. This value can be configured with the result-size parameter in the query string of the HTTP request."
"phone"
"blender"
"tv"
"socks"
"broiler"
"shirt"
"toaster"
"phone"
"blender"
"tv"
"socks"
"broiler"
"shirt"
"toaster"
"phone"
"blender"
"tv"
"socks"
"broiler"
"shirt"
"toaster"
"phone"
"blender"
"tv"
"socks"
"broiler"
"shirt"
"toaster"
"phone"
"blender"
"tv"
"socks"
"broiler"
"shirt"
"toaster"
"phone"
"blender"
"tv"
"socks"
"broiler"
"shirt"
"toaster"
"phone"
"blender"
"tv"
"socks"
"broiler"
"shirt"
"toaster"
"phone"
"blender"
"tv"
"socks"
"broiler"
"shirt"
"toaster"
"phone"
"blender"
"tv"
"socks"
"broiler"
"shirt"
"toaster"
"phone"
"blender"
"tv"
"socks"
"broiler"
"shirt"
"toaster"
"phone"
"blender"
"tv"
"socks"
"broiler"
"shirt"
"toaster"
"phone"
"blender"
"tv"
"socks"
"broiler"
"shirt"
"toaster"
"phone"
"blender"
"tv"
"socks"
"broiler"
"shirt"
"toaster"
"phon

## Order by clauses

Order by clauses are used to reorganize the order of the tuples, but without altering them. They are the same as ORDER BY clauses in SQL.

In [71]:
%%jsoniq
for $x in -2 to 2
let $square := $x * $x
order by $square
return {
  "number": $x,
  "square": $square
}

Took: 0.01975703239440918 ms
{"number": 0, "square": 0}
{"number": -1, "square": 1}
{"number": 1, "square": 1}
{"number": -2, "square": 4}
{"number": 2, "square": 4}


It is also possible, like in SQL, to specify an ascending or a descend- ing order. By default, the order is ascending.


In [72]:
%%jsoniq
for $x in -2 to 2
let $square := $x * $x
order by $square ascending
return {
  "number": $x,
  "square": $square
}

Took: 0.013797283172607422 ms
{"number": 0, "square": 0}
{"number": -1, "square": 1}
{"number": 1, "square": 1}
{"number": -2, "square": 4}
{"number": 2, "square": 4}


In [74]:
%%jsoniq
for $x in -2 to 2
let $square := $x * $x
order by $square descending
return {
  "number": $x,
  "square": $square
}

Took: 0.01792001724243164 ms
{"number": -2, "square": 4}
{"number": 2, "square": 4}
{"number": -1, "square": 1}
{"number": 1, "square": 1}
{"number": 0, "square": 0}


In case of ties between tuples, the order is arbitrary. But it is possible to sort on another variable in case there is a tie with the first one (compound sorting keys):

In [75]:
%%jsoniq
for $x in -2 to 2
let $square := $x * $x
order by $square descending, $x ascending return {
   "number": $x,
   "square": $square
 }


Took: 0.014029979705810547 ms
{"number": -2, "square": 4}
{"number": 2, "square": 4}
{"number": -1, "square": 1}
{"number": 1, "square": 1}
{"number": 0, "square": 0}


It is possible to control what to do with empty sequences: they can be considered smallest or greatest.

In [76]:
%%jsoniq
for $x in 1 to 5
let $y := $x[$$ mod 2 = 1]
order by $y ascending empty greatest
return [ $y ]


Took: 0.014728307723999023 ms
[1]
[3]
[5]
[]
[]


In [77]:
%%jsoniq
for $x in 1 to 5
let $y := $x[$$ mod 2 = 1]
order by $y ascending empty least
return [ $y ]

Took: 0.013755083084106445 ms
[]
[]
[1]
[3]
[5]


Here is another example of use of the order by clause with our datasets:

In [78]:
%%jsoniq
for $product in json-file("http://www.rumbledb.org/samples/products-small.json")
let $store := json-file("http://www.rumbledb.org/samples/countries.json")
                [$$.sid eq $product.store-number]
order by count($store) descending,
         string-length($product.product) ascending
return $product.product

Took: 4.2881550788879395 ms
"Warning! The output sequence contains 100000 items but its materialization was capped at 200 items. This value can be configured with the result-size parameter in the query string of the HTTP request."
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"
"tv"

## Group by clauses

Group by clauses organize tuples in groups based on matching keys, and then output only one tuple for each group, aggregating other variables (count, sum, max, min...). This is similar to GROUP BY clauses in SQL.

In [83]:
%%jsoniq
for $x in 1 to 5
let $y := $x mod 2
group by $y
return {
  "grouping key" : $y,
  "count of x" : count($x)
}

Took: 0.01671290397644043 ms
{"grouping key": 0, "count of x": 2}
{"grouping key": 1, "count of x": 3}


However, JSONiq’s group by clauses are more powerful and expres- sive than SQL GROUP BY clauses: indeed, it is also possible to opt out of aggregating other (non-grouping-key) variables. Then, for a non- aggregated variable, the sequence of all its values of within a group will be rebound to this same variable as a single binding in the outcoming tuple. It is thus possible to write many more queries than SQL would allow, which is one of the reasons why a language like JSONiq should be preferred for nested datasets.


In [84]:
%%jsoniq
for $x in 1 to 5
let $y := $x mod 2
group by $y
return {
  "grouping key" : $y,
  "grouped x values" : [ $x ]
}

Took: 0.01664900779724121 ms
{"grouping key": 0, "grouped x values": [2, 4]}
{"grouping key": 1, "grouped x values": [1, 3, 5]}


Finally, here is an example of use of a group by clause with our example dataset.

In [87]:
%%jsoniq
for $product in json-file("http://www.rumbledb.org/samples/products-small.json")
group by $sid := $product.store-number
order by $sid
let $store := json-file("http://www.rumbledb.org/samples/countries.json")
                 [$$.sid eq $sid]
return {|
   $store,
   { "products" : [ distinct-values($product.product) ] }
|}

Took: 3.1400740146636963 ms
{"sid": 64, "country": "Egypt", "products": ["broiler", "shirt", "toaster", "phone", "blender", "tv", "socks"]}
{"sid": 68, "country": "Estonia", "products": ["shirt", "toaster", "phone", "blender", "tv", "socks", "broiler"]}
{"sid": 42, "country": "Central African Republic", "products": ["toaster", "phone", "blender", "tv", "socks", "broiler", "shirt"]}
{"sid": 83, "country": "Gibraltar", "products": ["blender", "tv", "socks", "broiler", "shirt", "toaster", "phone"]}
{"sid": 54, "country": "Cote D\"Ivoire", "products": ["tv", "socks", "broiler", "shirt", "toaster", "phone", "blender"]}
{"sid": 96, "country": "Holy See (Vatican City State)", "products": ["socks", "broiler", "shirt", "toaster", "phone", "blender", "tv"]}
{"sid": 82, "country": "Ghana", "products": ["blender", "tv", "socks", "broiler", "shirt", "toaster", "phone"]}
{"sid": 78, "country": "Gabon", "products": ["phone", "blender", "tv", "socks", "broiler", "shirt", "toaster"]}
{"sid": 41, "count